# Carga de datos

In [23]:
import os
import cv2
import numpy as np

def getImages(directorio, size=(256,256)):
    datos = []
    etiquetas = []
    for etiqueta, clase in enumerate(["fumador", "noFumador"]):
        pathClass = os.path.join(directorio, clase)
        for img in os.listdir(pathClass):
            imgPath = os.path.join(pathClass, img)
            imagen = cv2.imread(imgPath)
            if imagen is not None: 
                imagen = cv2.resize(imagen, size)
                datos.append(imagen)
                etiquetas.append(etiqueta)
    return (np.array(datos), np.array(etiquetas))

datos, etiquetas = getImages("data")

In [24]:
noFumadores = datos[etiquetas == 1]
fumadores = datos[etiquetas == 0]

print("No fumadores: ", noFumadores.shape)
print("Fumadores: ", fumadores.shape)

No fumadores:  (1276, 256, 256, 3)
Fumadores:  (1969, 256, 256, 3)


In [25]:
# Paso 2: Separar las clases
datos_fumadores = datos[etiquetas == 0]
etiquetas_fumadores = etiquetas[etiquetas == 0]

datos_no_fumadores = datos[etiquetas == 1]
etiquetas_no_fumadores = etiquetas[etiquetas == 1]

In [26]:
# Paso 3: Definir el número de muestras (tamaño mínimo de ambas clases)
n_samples = min(len(datos_fumadores), len(datos_no_fumadores))

In [27]:
# Paso 4: Reducir ambas clases al tamaño mínimo (undersampling)
from sklearn.utils import resample
datos_fumadores_reducidos, etiquetas_fumadores_reducidos = resample(
    datos_fumadores,
    etiquetas_fumadores,
    replace=False,
    n_samples=n_samples,
    random_state=42
)

In [28]:
datos_no_fumadores_reducidos, etiquetas_no_fumadores_reducidos = resample(
    datos_no_fumadores,
    etiquetas_no_fumadores,
    replace=False,
    n_samples=n_samples,
    random_state=42
)

In [29]:
# Paso 5: Concatenar los datos balanceados
datos_balanceados = np.concatenate((datos_fumadores_reducidos, datos_no_fumadores_reducidos))
etiquetas_balanceadas = np.concatenate((etiquetas_fumadores_reducidos, etiquetas_no_fumadores_reducidos))

In [30]:

# Paso 6: Verificar el tamaño de cada clase
noFumadores = datos_balanceados[etiquetas_balanceadas == 1]
fumadores = datos_balanceados[etiquetas_balanceadas == 0]

In [31]:
print("No fumadores: ", noFumadores.shape)
print("Fumadores: ", fumadores.shape)

No fumadores:  (1276, 256, 256, 3)
Fumadores:  (1276, 256, 256, 3)


In [32]:
datos_balanceados_normalizados = datos_balanceados / 255.0

# Dividir los datos

In [33]:
from sklearn.model_selection import train_test_split
# Dividir datos en entrenamiento y validación (80% entrenamiento, 20% validación)
X_train, X_val, y_train, y_val = train_test_split(
    datos_balanceados_normalizados, 
    etiquetas_balanceadas, 
    test_size=0.2, 
    random_state=42
)

In [34]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Cargar el modelo base preentrenado
base_model = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Congelar las capas del modelo base

# Añadir capas personalizadas para la clasificación binaria
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Definir el modelo
model = Model(inputs=base_model.input, outputs=predictions)

C:\Users\caste\AppData\Local\Temp\ipykernel_13380\3064595345.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [35]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
import tensorflow as tf

# Verificar si hay GPUs disponibles
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPU(s) disponible(s): {len(gpus)}")
else:
    print("No se detectaron GPUs.")

No se detectaron GPUs.


In [38]:
from tensorflow.keras.callbacks import EarlyStopping

# Definir un callback para detener el entrenamiento si no mejora
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 492ms/step - accuracy: 0.8153 - loss: 0.4104 - val_accuracy: 0.7828 - val_loss: 0.4495
Epoch 2/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 463ms/step - accuracy: 0.8651 - loss: 0.3172 - val_accuracy: 0.8121 - val_loss: 0.4074
Epoch 3/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 482ms/step - accuracy: 0.9061 - loss: 0.2321 - val_accuracy: 0.8082 - val_loss: 0.4078
Epoch 4/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step - accuracy: 0.9453 - loss: 0.1756

KeyboardInterrupt: 